In [ ]:
import os
import json
from openai import OpenAI
client = OpenAI()

client.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def get_current_weather(location, unit="섭씨"):
    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info, ensure_ascii=False)


In [ ]:
assistant = client.beta.assistants.create(
    name="날씨봇",
    instructions="당신은 날씨 챗봇입니다. 사용자의 질문에 대답하기 위해 주어진 function들을 사용하세요.",
    tools = [
        {
            "type": "function",
            "function":
            {
                "name": "get_current_weather",
                "description": "특정 지역의 날씨를 알려줍니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "지역이름. 예) 서울, 부산, 제주도",
                        },
                        "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
                    },
                    "required": ["location"],
                },
            }
        }
    ],
    model="gpt-3.5-turbo"
)
assistant

In [ ]:
thread = client.beta.threads.create()
thread

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="서울의 날씨는 어때?"
)

In [ ]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [ ]:
run_id = run.id

In [ ]:
import time

while True:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run_id
    )
    if run.required_action is not None:
        break

In [ ]:
print(run.required_action.submit_tool_outputs.tool_calls[0])

In [ ]:
import json

available_functions = {
    "get_current_weather": get_current_weather,
}

for tool_call in run.required_action.submit_tool_outputs.tool_calls:
    tool_call_id = tool_call.id
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args['location']
    )

print(function_response)

In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
      {
        "tool_call_id": tool_call_id,
        "output": json.dumps(function_response, ensure_ascii=False)
      }
    ]
)
run_id = run.id

In [ ]:
import time

while run.status != "completed"  :
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    time.sleep(0.1)

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [ ]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data[0].content[0].text.value)